### Advanced SQL

In [ ]:
import pandas as pd
%load_ext sql
%sql sqlite://

In [ ]:
# For compatibility across multiple platforms
import os
IB = os.environ.get('INSTABASE_URI',None) is not None
open = ib.open if IB else open

In [ ]:
# Load tables from CSV files
# Cities
with open('Cities.csv','rU') as f:
    Cities = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Cities;
%sql persist Cities
# Countries
with open('Countries.csv','rU') as f:
    Countries = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Countries;
%sql persist Countries
# Players
with open('Players.csv','rU') as f:
    Players = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Players;
%sql persist Players
# Teams
with open('Teams.csv','rU') as f:
    Teams = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Teams;
%sql persist Teams

#### Look at sample of Cities and Countries tables

In [ ]:
%%sql
select * from Cities limit 5

In [ ]:
%%sql
select * from Countries limit 5

### Duplicates, table variables

*Warm-up: Find all cities in the EU with temperature > 15*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

*Modify previous query to: (1) return country instead of city (2) remove duplicates (3) change to non-EU (4) use table variables*

*Find all pairs of cities with the same longitude; return the city pairs and their (shared) longitude - notice what's wrong and fix it*

In [ ]:
%%sql
select C1.city, C2.city, C1.longitude
from Cities C1, Cities C2
where C1.longitude = C2.longitude

*Find all pairs of cities that are near each other, i.e., lat and lng are both less than 0.5 apart; return city pairs*

In [ ]:
%%sql
select C1.city, C2.city
from Cities C1, Cities C2
where abs(C1.longitude - C2.longitude) < .5
and abs(C1.latitude - C2.latitude) < .5
and C1.city < C2.city

### <font color = 'green'>Your Turn</font>

*Find all pairs of cities with the same temperature. Return the city pairs along with their shared temperature. What do you think about the data after seeing the answer?*

In [ ]:
%%sql
FILL IN QUERY HERE

### Subqueries in Where clause

*Find all countries in the Countries table with no city in the Cities table*

In [ ]:
%%sql
select country
from Countries
where not exists (select city from Cities
                  where Cities.country = Countries.country)

*Find countries in the EU that have a city with temperature > 15*

In [ ]:
%%sql
select country
from Countries
where EU = 'yes'
and exists (select * from Cities
            where Cities.country = Countries.country
            and temperature > 15)

*Find the westernmost city; return the city and longitude*

In [ ]:
%%sql
select city, longitude
from Cities C1
where not exists (select * from Cities C2
                  where C2.longitude < C1.longitude)

*Add easternmost to previous query*

*Westernmost city query using = and min*

In [ ]:
%%sql
select city, longitude
from Cities
where longitude = (select min(longitude) from Cities)

*Find all cities whose temperature is more than 50% higher than the average; return the city, country, and temperature, ordered by descending temperature*

In [ ]:
%%sql
select city, country, temperature
from Cities
where temperature > (select avg(temperature) * 1.5 from Cities)
order by temperature desc

*Number of cities in the EU*

In [ ]:
%%sql
select count()
from Cities
where country in (select country from Countries where EU = 'yes')

*Modify previous query to use "not in"*

*Same query using join instead of subquery*

In [ ]:
%%sql
select count()
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes'

*Number of countries having cities with temperature < 12*

In [ ]:
%%sql
select count()
from Countries
where country in (select country from Cities where temperature < 12)

*Same query using join instead of subquery*

In [ ]:
%%sql
select count()
from Countries, Cities
where Cities.country = Countries.country
and temperature < 12

*Find countries in Countries table with no city in Cities table using join instead of subquery (subquery version repeated first)*

In [ ]:
%%sql
select country
from Countries
where not exists (select city from Cities
                  where Cities.country = Countries.country)

In [ ]:
%%sql
FILL IN

### <font color = 'green'>Your Turn</font>

*Find all cities in a country whose population is < 2; return the city and country. First write the query without a subquery.*

In [ ]:
%%sql
YOUR QUERY HERE

*Now write the same query using a subquery instead of a join.*

In [ ]:
%%sql
FILL IN QUERY HERE

*Find all countries with no city having a temperature > 6*

In [ ]:
%%sql
FILL IN QUERY HERE

*Now try to write the same query without a subquery; can you?*

In [ ]:
%%sql
FILL IN QUERY HERE

### Aggregation with Having clause

*Find all countries with average city temperature > 10; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > 10

*Find all countries with more than 10 cities*

In [ ]:
%%sql
select country
from Cities
group by country
having count() > 10

*Same query without Having clause*

In [ ]:
%%sql
select distinct country
from Cities C1
where 10 < (select count() from Cities C2
            where C1.country=C2.country)

*Which combinations of EU versus non-EU and coastline versus no-coastline have a minimum population greater than 0.5?*

In [ ]:
%%sql
select EU, coastline, min(population)
from Countries
group by EU, coastline
having min(population) > 0.5

*Find all countries with average city temperature more than 50% higher than the overall average; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > (select 1.5 * avg(temperature) from Cities)

### <font color = 'green'>Your Turn</font>

*Find all countries whose average city longitude is lower than the overall average longitude, and whose average city latitude is higher than the overall average latitude. Return the countries. Note: Yes, you can use "and" in Having clauses!*

In [ ]:
%%sql
FILL IN QUERY HERE

### Subqueries in From and Select clauses

*Find all countries with both cold and warm cities -- at least one city with temperature < 9 and one city with temperature > 14*

In [ ]:
%%sql
select C1.country
from Cities C1, Cities C2
where C1.country = C2.country
and C1.temperature < 9 and C2.temperature > 14

*Modify query to also return count of cold and warm cities*

In [ ]:
# (select count() from Cities where country = C1.country and temperature < 9) as numcold,
# (select count() from Cities where country = C1.country and temperature > 14) as numwarm

*Same query using subquery in From clause instead of Select clause*

In [ ]:
%%sql
select Cold.country, numcold, numwarm
from (select country, count() as numcold from Cities
      where temperature < 9 group by country) Cold,
     (select country, count() as numwarm from Cities
      where temperature > 14 group by country) Warm
where Cold.country = Warm.country

### Data modification

*Increase all city temperatures by 10%*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature

In [ ]:
%%sql
select avg(temperature) from Cities

*Increase temperatures another 10% for cities in countries with coastline*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature
where country in (select country from Countries
                  where coastline = 'yes')

*Delete all cities in Turkey*

In [ ]:
%%sql
delete from Cities
where country = 'Turkey'

*Create a new table non-EU containing list of cities (with country) not in the EU*

In [ ]:
%%sql
drop table if exists NonEU;
create table NonEU(city, country);
insert into NonEU
  select city, country from cities
  where country in (select country from Countries
                    where EU = 'no');
select * from NonEU

*Add your city*

In [ ]:
%%sql
insert into NonEU values ('my-city','my-country');
select * from NonEU

### <font color = 'green'>Your Turn - Advanced SQL on World Cup Data</font>

#### Look at sample of Players and Teams tables

In [ ]:
%%sql
select * from Players limit 5

In [ ]:
%%sql
select * from Teams limit 5

*1) Find all pairs of teams who have the same number of goalsFor as
each other and the same number of goalsAgainst as each other.
Return the teams and numbers of goalsFor and goalsAgainst.
Make sure to return each pair only once.*

In [ ]:
%%sql
FILL IN QUERY HERE

*2) Find all teams with ranking <30 where no player made more than 150 passes. Return the team and ranking.*

In [ ]:
%%sql
FILL IN QUERY HERE

*3) Which team has the highest ratio of goalsFor to goalsAgainst?*

In [ ]:
%%sql
FILL IN QUERY HERE

*4) Find all team-position pairs where the average number of passes made by players in that position on that team is greater than 150. Return the team-position pairs.*

In [ ]:
%%sql
FILL IN QUERY HERE

*5) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [ ]:
%%sql
FILL IN QUERY HERE

*6) Which team has the highest average number of passes per minute played? Return the team and average passes per minute. Note: To force floating point division, multiply one operand by 1.0. Hints: (1) You can compute a team's average number of passes per minute played by dividing the total number of passes by the total number of minutes. (2) Consider using Limit.*

In [ ]:
%%sql
FILL IN QUERY HERE